2.1 Preprocessing

In [4]:
import pandas as pd
import numpy as np
import logging
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
import logging
import os

current_directory = os.getcwd()
log_file_path = os.path.join(current_directory, 'task_1.log')
logging.basicConfig(filename=log_file_path, level=logging.INFO, 
                    format='%(asctime)s - %(levelname)s - %(message)s')

train = pd.read_csv('../../doc4/rossmann-store-sales/train.csv', parse_dates=['Date'])
store = pd.read_csv('../../doc4/rossmann-store-sales/store.csv')

train_merged = pd.merge(train, store, on='Store', how='left')
logging.info('Merged train and store datasets.')


imputer = SimpleImputer(strategy='median')
train_merged['CompetitionDistance'] = imputer.fit_transform(train_merged[['CompetitionDistance']])
train_merged['Promo2SinceYear'] = imputer.fit_transform(train_merged[['Promo2SinceYear']])
train_merged.fillna(0, inplace=True)  
logging.info('Handled missing values in the dataset.')


train_merged['Day'] = train_merged['Date'].dt.day
train_merged['Month'] = train_merged['Date'].dt.month
train_merged['Year'] = train_merged['Date'].dt.year
train_merged['WeekOfYear'] = train_merged['Date'].dt.isocalendar().week
train_merged['Weekday'] = train_merged['Date'].dt.weekday
train_merged['Weekend'] = train_merged['Weekday'].apply(lambda x: 1 if x >= 5 else 0)


train_merged['IsHoliday'] = np.where((train_merged['StateHoliday'] != '0'), 1, 0)
train_merged['DaysToHoliday'] = (pd.to_datetime('2023-12-25') - train_merged['Date']).dt.days
train_merged['StartMidEndMonth'] = pd.cut(train_merged['Day'], bins=[0, 10, 20, 31], labels=['Start', 'Mid', 'End'])

logging.info('Extracted features from date columns.')


train_merged = pd.get_dummies(train_merged, columns=['StoreType', 'Assortment', 'PromoInterval', 'StateHoliday', 'StartMidEndMonth'], drop_first=True)
logging.info('Converted categorical columns to numeric.')


features = train_merged.drop(['Sales', 'Customers', 'Date'], axis=1)
target = train_merged['Sales']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(features)
logging.info('Scaled features using StandardScaler.')


X_train, X_test, y_train, y_test = train_test_split(X_scaled, target, test_size=0.2, random_state=42)
logging.info('Performed train-test split.')

C:\Users\USER\AppData\Local\Temp\ipykernel_26068\1626751758.py:15: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv('../../doc4/rossmann-store-sales/train.csv', parse_dates=['Date'])
